In [41]:
import functools
import math
import pickle
import random

### Get a sieve of all numbers $n$ to indicate whether they are prime, with $0 \leq n \leq 10^7$

In [40]:
def get_primes_eratosthenes_sieve(n):
    is_prime = [1] * (n + 1)
    p = 2
    is_prime[0] = 0
    is_prime[1] = 0
    while p * p <= n:
        if is_prime[p]:
            for i in range(p * 2, n + 1, p):
                is_prime[i] = 0
        p += 1
    prime_list = []
    for i in range(n + 1):
        if is_prime[i]:
            prime_list.append(i)
    return is_prime

is_prime = get_primes_eratosthenes_sieve(10 ** 7)

### Use Pollard's Rho algorithm for finding prime factorizations, and thereby the list of all divisors, for any $n$

In [42]:
def fast_miller_rabin(n, use_probabilistic=False, tolerance=30):
    """
    Tests whether a number is prime using a deterministic version of the Miller-
    Rabin primality test. Optionally tests whether the specified number is a
    prime probabilistically up to a given tolerance using the regular version of
    the Miller-Rabin test. If the number is greater than 10^36, then all witnesses
    in the range [2, 2*log(n)*log(log(n))] are tested. However, this is conjectural
    and only heuristic evidence exists for it. To certify that a number is actually
    prime, one needs to test all witnesses in the range [2, 2*log(n)^2]. However,
    this is generally quite slow.
    Arguments:
        n (:int) - the integer to be tested
        use_probabilistic (:bool) - flag to indicate whether to use the regular
                                   version of the Miller-Rabin primality test
        tolerance (:int) - number of trials to be used to test primality
    Returns:
        True if 'n' is prime (or probably prime) and False otherwise
    References:
        - Francky from the PE Forums
        - https://miller-rabin.appspot.com/
        - https://en.wikipedia.org/wiki/Miller-Rabin_primality_test
    """
    firstPrime = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]
    # Determine bases for deterministic Miller-Rabin test
    if n >= 10 ** 36:
        log_n = math.log(n)
        if not use_probabilistic:
            w = range(2, 2 * int(log_n * math.log(log_n) / math.log(2)))
        else:
            w = range(tolerance)
    elif n >= 1543267864443420616877677640751301:
        w = firstPrime[:20]
    elif n >= 564132928021909221014087501701:
        w = firstPrime[:18]
    elif n >= 59276361075595573263446330101:
        w = firstPrime[:16]
    elif n >= 6003094289670105800312596501:
        w = firstPrime[:15]
    elif n >= 3317044064679887385961981:
        w = firstPrime[:14]
    elif n >= 318665857834031151167461:
        w = firstPrime[:13]
    elif n >= 3825123056546413051:
        w = firstPrime[:12]
    # [2, 3, 5, 7, 11, 13, 17, 19, 23]
    elif n >= 341550071728321:
        w = firstPrime[:9]
    # [2, 3, 5, 7, 11, 13, 17]
    elif n >= 3474749660383:
        w = firstPrime[:7]
    elif n >= 2152302898749:
        w = firstPrime[:6]
    # [2, 3, 5, 7, 11, 13]
    elif n >= 4759123141:
        w = firstPrime[:5]
    # [2, 3, 5, 7, 11]
    elif n >= 9006403:
        w = [2, 7, 61]
    elif n >= 489997:
        # Some Fermat stuff
        if n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43 and n % 47 and n % 53 and n % 59 and n % 61 and n % 67 and n % 71 and n % 73 and n % 79 and n % 83 and n % 89 and n % 97 and n % 101:
            hn, nm1 = n >> 1, n - 1
            p = pow(2, hn, n)
            if p == 1 or p == nm1:
                p = pow(3, hn, n)
                if p == 1 or p == nm1:
                    p = pow(5, hn, n)
                    return p == 1 or p == nm1
        return False
    elif n >= 42799:
        return n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43 and pow(2, n - 1, n) == 1 and pow(5, n - 1, n) == 1
    elif n >= 841:
        return n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43 and n % 47 and n % 53 and n % 59 and n % 61 and n % 67 and n % 71 and n % 73 and n % 79 and n % 83 and n % 89 and n % 97 and n % 101 and n % 103 and pow(2, n - 1, n) == 1
    elif n >= 25:
        return n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23
    elif n >= 4:
        return n & 1 and n % 3
    else:
        return n > 1
    if not (n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17
            and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43
            and n % 47 and n % 53 and n % 59 and n % 61 and n % 67 and n % 71 and n % 73
            and n % 79 and n % 83 and n % 89):
        return False
    # Miller-Rabin
    s = 0
    d = n - 1
    while not d & 1:
        d >>= 1
        s += 1
    for k in w:
        # Pick a random witness if probabilistic
        if use_probabilistic:
            p = random.randint(2, n - 2)
        else:
            p = k
        x = pow(p, d, n)
        if x == 1:
            continue
        for _ in range(s):
            if x + 1 == n:
                break
            x = x * x % n
        else:
            return False
    return True


# https://github.com/zhangbo2008/python_algorithm2/blob/c53669703b957a079f100c12711f86f5fc2f9389/algorithms/factorization/pollard_rho.py
def pollard_rho_prime_factorization(x):
    def f(x):
        return x * x + 1

    def rho(n, x1=2, x2=2):
        if n % 2 == 0:
            return 2
        i = 0
        while True:
            x1 = f(x1) % n
            x2 = f(f(x2)) % n
            divisor = math.gcd(abs(x1 - x2), n)
            i += 1
            if divisor != 1:
                break
            if i > 500:
                x1 = random.randint(1, 10)
                x2 = random.randint(1, 10)
                i = 0
        return divisor

    def pollard_rho_rec(x, factors):
        if x == 1:
            return

        if x <= 10 ** 7:
            if is_prime[x]:
                factors.append(x)
                return
        elif fast_miller_rabin(x):
            factors.append(x)
            return

        divisor = rho(x, random.randint(1, 10), random.randint(1, 10))
        pollard_rho_rec(divisor, factors)
        pollard_rho_rec(x // divisor, factors)

    if x == 1 or x == 0:
        return dict()
    factors = []
    pollard_rho_rec(x, factors)
    return [(f, factors.count(f)) for f in set(factors)]

def divisor_list_from_factors(factors):
    # https://stackoverflow.com/questions/171765/what-is-the-best-way-to-get-all-the-divisors-of-a-number
    num_factors = len(factors)
    f = [0] * num_factors
    while True:
        yield functools.reduce(lambda x, y: x * y, [factors[x][0] ** f[x] for x in range(num_factors)], 1)
        i = 0
        while True:
            f[i] += 1
            if f[i] <= factors[i][1]:
                break
            f[i] = 0
            i += 1
            if i >= num_factors:
                return

### Retrieve prime factorizations for each $n$, with $1\leq n\leq 10^8$, pre-computed using Pollard's Rho algorithm  

In [34]:
with open('..\\Computation Caches\\prime_factorizations_1_100000000.pkl', 'rb') as file:
    prime_factorization = pickle.load(file)

### From [A147811](https://oeis.org/A147811): Alexandrian integers are numbers of the form $p\cdot(p+d)\cdot\left(p+\frac{(p^2+1)}{d}\right)$, where $d\mid p^2+1$.

In [50]:
L = 10 ** 8
n_max = 150000
alexandrian_numbers = set()
for p in range(1, 10 ** 5 + 1):
    divisor_list_p = divisor_list_from_factors(list(prime_factorization[p ** 2 + 1].items()) if p ** 2 + 1 <= L else pollard_rho_prime_factorization(p ** 2 + 1))
    for d in divisor_list_p:
        alexandrian_numbers.update({p * (p + d) * (p + (p ** 2 + 1) // d)})
    
alexandrian_numbers = sorted(alexandrian_numbers)
alexandrian_numbers[n_max - 1]

1884161251122450